1) **Derives a new table of US Customers called USACusts.** notice here we're using the derived table recieved from a subquery in our FROM clause.

In [2]:
use Northwinds2022TSQLV7

select *
from (
    select Customerid, CustomerCompanyName
    from Sales.Customer
    where CustomerCountry = N'USA'
) as USACusts

(13 rows affected)

Total execution time: 00:00:00.018

Customerid,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


2) **Derives a table for each years number of customers.** Makes use of inline aliasing, take a peek at comments.

In [13]:
use Northwinds2022TSQLV7

-- select YEAR(OrderDate) as orderyear,
--     COUNT(distinct CustomerId) as numcusts
-- from Sales.[Order]
-- group by orderyear --obv we'll get a parsing error

select orderyear,
    COUNT(distinct custid) as numcusts
from (
    --years each customer has ordered
    select YEAR(OrderDate) as orderyear,
        CustomerId as custid --these lines will execute before our first statement so hence the aliasing is possible
    from Sales.[Order]
) as D
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.026

orderyear,numcusts
2014,67
2015,86
2016,81


3) **External column aliasing, does same as above.**

In [16]:
use Northwinds2022TSQLV7

select orderyear, count(distinct custid) as numcusts
from (
    select year(OrderDate), CustomerId --notice we dont alias here
    from Sales.[Order] 
) as D (orderyear, custid) -- but here after we return the derived table!
group by orderyear

(3 rows affected)

Total execution time: 00:00:00.046

orderyear,numcusts
2014,67
2015,86
2016,81


4) **Yearly count of customers handled by Employee 3.** Uses arguments.

In [20]:
use Northwinds2022TSQLV7

DECLARE @empid AS INT = 3; --used as an argument for the table we derive

SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM (SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
      FROM Sales.[Order]
      WHERE EmployeeId = @empid) AS D
GROUP BY orderyear;
GO

(3 rows affected)

Total execution time: 00:00:00.017

orderyear,numcusts
2014,16
2015,46
2016,30


5) **Years where number of customers exceeds 70.** Using nested derived tables for inline aliasing, notice how the top layer is easiest to read.

In [21]:
use Northwinds2022TSQLV7

SELECT orderyear, numcusts
FROM (SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
      FROM (SELECT YEAR(OrderDate) AS orderyear, 
                CustomerId as custid
            FROM Sales.[Order]) AS D1
      GROUP BY orderyear) AS D2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.033

orderyear,numcusts
2015,86
2016,81


6) **Same as above, but not as readable as you can see due to top level aliasing.**

In [22]:
use Northwinds2022TSQLV7

SELECT YEAR(OrderDate) AS orderyear, COUNT(DISTINCT CustomerId) AS numcusts
FROM Sales.[Order]
GROUP BY YEAR(OrderDate)
HAVING COUNT(DISTINCT CustomerId) > 70;

(2 rows affected)

Total execution time: 00:00:00.022

orderyear,numcusts
2015,86
2016,81


7. **Number of current customers, previous customer, and growth of customers for each year.** Using two subquery to create tables of current customers, previous customers, uses left join to combine row with certain criteria, preserves NULLS.

In [31]:
use Northwinds2022TSQLV7

SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM (SELECT YEAR(OrderDate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(OrderDate)) AS Cur
  LEFT OUTER JOIN --we use left join to preserve rows and nulls, so as to not discard
     (SELECT YEAR(OrderDate) AS orderyear,
        COUNT(DISTINCT CustomerId) AS numcusts
      FROM Sales.[Order]
      GROUP BY YEAR(OrderDate)) AS Prv
    ON Cur.orderyear = Prv.orderyear + 1;


(3 rows affected)

Total execution time: 00:00:00.036

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


8. **Create USACusts as derived table using COMMON table expressions.**

In [24]:
use Northwinds2022TSQLV7;

WITH USACusts AS
(
  SELECT CustomerId, CustomerCompanyName
  FROM Sales.Customer
  WHERE CustomerCountry = N'USA'
)
SELECT * FROM USACusts;

(13 rows affected)

Total execution time: 00:00:00.027

CustomerId,CustomerCompanyName
32,Customer YSIQX
36,Customer LVJSO
43,Customer UISOJ
45,Customer QXPPT
48,Customer DVFMB
55,Customer KZQZT
65,Customer NYUHS
71,Customer LCOUJ
75,Customer XOJYP
77,Customer LCYBZ


9. **Yearly count of customers.** Table expressions and INLINE column aliasing.

In [26]:
use Northwinds2022TSQLV7;

WITH C AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.030

orderyear,numcusts
2014,67
2015,86
2016,81


10) **Yearly count of customers.** Table expression and EXTERNAL column aliasing.

In [27]:
use Northwinds2022TSQLV7;

WITH C(orderyear, custid) AS
(
  SELECT YEAR(OrderDate), CustomerId
  FROM Sales.[Order]
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.031

orderyear,numcusts
2014,67
2015,86
2016,81


11) **Yearly count of customers handled by employee 3.** Table expressions, inline column aliasing, and ARGUMENTS.

In [28]:
use Northwinds2022TSQLV7;

DECLARE @empid AS INT = 3;

WITH C AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
        CustomerId as custid
  FROM Sales.[Order]
  WHERE EmployeeId = @empid
)
SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
FROM C
GROUP BY orderyear;

(3 rows affected)

Total execution time: 00:00:00.034

orderyear,numcusts
2014,16
2015,46
2016,30


12. Multiple CTES. High readibility. Creates a table of order years for each customer. Derives a table with column aliasing, we use this in another CTE. Derivies a table of the number of unique customers for each year. We use the final derived table to easily return **years where the cumber of customers exceeds 70.**

In [33]:
use Northwinds2022TSQLV7;

WITH C1 AS
(
  SELECT YEAR(OrderDate) AS orderyear, 
    CustomerId as custid --readable aliasing
  FROM Sales.[Order]
),
C2 AS
(
  SELECT orderyear, COUNT(DISTINCT custid) AS numcusts
  FROM C1
  GROUP BY orderyear
)
SELECT orderyear, numcusts
FROM C2
WHERE numcusts > 70;

(2 rows affected)

Total execution time: 00:00:00.033

orderyear,numcusts
2015,86
2016,81


13. Utilizing MULTIPLE references to **get each years current customers, previous customer, and growth.** Notice how the already named derived table can have more than one alias, to perform table operations like join.

In [35]:
use Northwinds2022TSQLV7;

WITH YearlyCount AS
(
  SELECT YEAR(OrderDate) AS orderyear,
    COUNT(DISTINCT CustomerId) AS numcusts
  FROM Sales.[Order]
  GROUP BY YEAR(OrderDate)
)
SELECT Cur.orderyear, 
  Cur.numcusts AS curnumcusts, Prv.numcusts AS prvnumcusts,
  Cur.numcusts - Prv.numcusts AS growth
FROM YearlyCount AS Cur -- the derived table has muliple references / aliases
  LEFT OUTER JOIN YearlyCount AS Prv 
    ON Cur.orderyear = Prv.orderyear + 1;

(3 rows affected)

Total execution time: 00:00:00.060

orderyear,curnumcusts,prvnumcusts,growth
2014,67,NULL,NULL
2015,86,67,19
2016,81,86,-5


14)

In [37]:
use Northwinds2022TSQLV7;

WITH EmpsCTE AS
(
  SELECT EmployeeId as empid, 
        EmployeeManagerId as  mgrid, 
        EmployeeFirstName as firstname,
        EmployeeLastName as lastname
  FROM HumanResources.Employee
  WHERE EmployeeId = 2
  
  UNION ALL --union the two tables, unioning a table to an operation including itself!
  
  --use employee2 table and employees table to derive a new one
  SELECT C.EmployeeId, C.EmployeeManagerId, C.EmployeeFirstName, C.EmployeeLastName
  FROM EmpsCTE AS P
    INNER JOIN HumanResources.Employee AS C
      ON C.EmployeeManagerId = P.empid
)
SELECT empid, mgrid, firstname, lastname
FROM EmpsCTE;

(8 rows affected)

Total execution time: 00:00:00.025

empid,mgrid,firstname,lastname
2,1,Don,Funk
3,2,Judy,Lew
5,2,Sven,Mortensen
6,5,Paul,Suurs
7,5,Russell,King
9,5,Patricia,Doyle
4,3,Yael,Peled
8,3,Maria,Cameron
